In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.init import kaiming_normal_, orthogonal_
import torchvision.models as models
from torch.nn.functional import threshold, normalize

In [ ]:
from glob import glob
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
import os
import cv2
import random
import time
import math

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" #this is to set the GPU to be used
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #if we have multiple GPUs, we can use 0,1 as well. Comment if no GPU available

In [ ]:
X = dumpman.undumper('list1.pkl') #list of file names for input
y = dumpman.undumper('list2.pkl') #list of file names for Ground Truth

X_train, X_rem, y_train, y_rem = train_test_split(X, y, test_size=0.05, random_state=42)


X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.5, random_state=42)

In [ ]:
print(len(X_train))
print(len(X_valid))

In [ ]:
fname = './pytorch'

batch = 4
num_epochs = 35

In [ ]:
img_row = 1024 #change these values to the size you need for your network
img_col = 1024 #network size depends on this as well
channels = 1
img_shape = (img_row,img_col,channels)

In [ ]:
use_cuda = True

In [ ]:
def encode_half(filters_in,filters_out,bn=True,drop=True): #an example of how to create functions for use with the DL Network you will use
    if bn and drop:
        return nn.Sequential(
            nn.Conv2d(filters_in, filters_out, kernel_size=3, stride=2, padding=1, bias=False),
            nn.ReLU(0.1),
            nn.BatchNorm2d(filters_out),            
            nn.Dropout(0.5)
        )
    if (bn==True and drop==False):
        return nn.Sequential(
            nn.Conv2d(filters_in, filters_out, kernel_size=3, stride=2, padding=1, bias=False),
            nn.ReLU(0.1),
            nn.BatchNorm2d(filters_out)
        )
    if (bn==False and drop==True):
        return nn.Sequential(
            nn.Conv2d(filters_in, filters_out, kernel_size=3, stride=2, padding=1, bias=False),
            nn.ReLU(0.1),           
            nn.Dropout(0.5)
        )
    if (bn==False and drop==False):
        return nn.Sequential(
            nn.Conv2d(filters_in, filters_out, kernel_size=3, stride=2, padding=1, bias=False),
            nn.ReLU(0.1)
        )

In [ ]:
class DeepLearning(nn.Module):
    def __init__(self):
        super(DeepLearning,self).__init__()
        
        # CNN  
        self.inp = nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1, bias=False) 
        
        self.enc = encode_half(1,3) #input size is 1 and output is 3

        self.op2 = nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1) #again 3 to 3
        #sigmoid
        self.sigmoid = nn.Sigmoid() #needed to normalize if you are working with images
        
    def forward(self, x, op1): 
 
        x = self.inp(x)        
        x = self.enc(x)                
        x = self.op(x)
        op = self.sigmoid(x)        
        
        return op #returns the output  

    def get_loss(self, input_value, output_value, opt):
        
        # define your loss funciton
        loss = torch.nn.functional.mse_loss(output_value, input_value) #we use MSE here and show how to calculate the loss given an output and an input
        
        return loss

    def step(self, inp, op, optimizer):
        
        optimizer.zero_grad() #this is needed to make optimizer zero for each epoch, so that the network learns new values
        loss = self.get_loss(inp, op) #calls the loss funciton and calculates loss
        loss.backward() #creates the backpropagation graph linking everything
        optimizer.step() #creates a step function which would save the whole generated graph
        
        return loss

In [ ]:
# Model
M_DL = DeepLearning() #creates an instance of the Deep Learning model
device = "cpu" #creates the default location

if use_cuda: #depending on the flag, the location where the model resides is changed
    print('CUDA used.')
    M_DL = M_DL.to("cuda")
    device = "cuda"
    
optimizer = torch.optim.Adam(M_DL.parameters(), lr=0.1, betas=(0.9, 0.999)) #creates our optimizer with whatever default value we need

In [ ]:
class My_Generator(torch.utils.data.Dataset): #this will be our training generator

    def __init__(self, input_filenames, output_filenames, batch_size= batch):
        self.input_filenames, self.output_filenames = input_filenames, output_filenames #this copies the values to local variables for our immediate use
        self.batch_size = batch_size
        
    def __len__(self):
        return np.uint16(np.ceil(len(self.input_filenames) / float(self.batch_size))) #since the number of input and output files are same, we can use either

    def __getitem__(self, idx):
        batch_x = self.input_filenames[idx * self.batch_size:(idx + 1) * self.batch_size] #creates the batches of file names
        batch_y = self.output_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        inp = np.zeros((len(batch_x),1,img_row,img_col),dtype=np.float32) #we go from a 1 layer input, like a grayscale image        
        op = np.zeros((len(batch_y),3,img_row,img_col),dtype=np.float32) #to a 3 layer output like RGB, for something like colorization
            
        for i in range(len(batch_x)): 
            inp[i,:,:,0] = cv2.resize(cv2.imread(batch_x[i], 0),(img_col,img_row)),(img_col,img_row)
            #here we read the file name in grayscale (0 flag in opencv), then resize it to our desired shape for DL
            #and then copy the values into our blank input array
            
            op[i] = np.reshape(cv2.resize(cv2.imread(batch_z[i]),(img_col,img_row)),(3,img_col,img_row))
            #we do the same thing here, but since the files are 3 layered, we resize and then reshape since 
            #opencv has the shape as col,row,3 while PyTorch uses 3,col,row
        
        return (inp/255, op/255) #normalize to 0-1 values for our use case

In [ ]:
class My_val_Generator(torch.utils.data.Dataset): #validation generator with the same things. but different name

    def __init__(self, input_filenames, output_filenames, batch_size= batch):
        self.input_filenames, self.output_filenames = input_filenames, output_filenames #this copies the values to local variables for our immediate use
        self.batch_size = batch_size
        
    def __len__(self):
        return np.uint16(np.ceil(len(self.input_filenames) / float(self.batch_size))) #since the number of input and output files are same, we can use either

    def __getitem__(self, idx):
        batch_x = self.input_filenames[idx * self.batch_size:(idx + 1) * self.batch_size] #creates the batches of file names
        batch_y = self.output_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        inp = np.zeros((len(batch_x),1,img_row,img_col),dtype=np.float32) #we go from a 1 layer input, like a grayscale image        
        op = np.zeros((len(batch_y),3,img_row,img_col),dtype=np.float32) #to a 3 layer output like RGB, for something like colorization
            
        for i in range(len(batch_x)): 
            inp[i,:,:,0] = cv2.resize(cv2.imread(batch_x[i], 0),(img_col,img_row)),(img_col,img_row)
            #here we read the file name in grayscale (0 flag in opencv), then resize it to our desired shape for DL
            #and then copy the values into our blank input array
            
            op[i] = np.reshape(cv2.resize(cv2.imread(batch_z[i]),(img_col,img_row)),(3,img_col,img_row))
            #we do the same thing here, but since the files are 3 layered, we resize and then reshape since 
            #opencv has the shape as col,row,3 while PyTorch uses 3,col,row
        
        return (inp/255, op/255) #normalize to 0-1 values for our use case

In [ ]:
my_training_batch_generator = My_Generator(X_train, y_trainbatch_size= batch)
my_validation_batch_generator = My_val_Generator(X_valid, y_valid, batch_size= batch)

In [ ]:
min_loss_t = 1e10 #base value for training
min_loss_v = 1e10 #base value for validation

M_DL.train() #indicates that we are using the train module instead of validation

for ep in range(num_epochs):
    
	st_t = time.time() #get current time
    
	print('='*50) #create a separation line so that it is easier to notice
    
	# Train
	M_DL.train() #configure for training the model

	loss_mean = 0 #we will add to this value, so it is zero
	loss_list = [] #cerate a blank list
    
	i = 0
	for x, y in my_training_batch_generator:
        
		x = torch.from_numpy(x) #configure these to numpy 
		y = torch.from_numpy(y)
        
		if use_cuda:       
			x = x.cuda(non_blocking=True) #load into cuda if configured
			y = y.cuda(non_blocking=True)
            
		ls = M_DL.step(x, y, optimizer) #this line is to send the input and output to the step funciton in the model
        
		if(math.isnan(float(ls))): #in case we go to undefined loss
			break
            
		loss_list.append(float(ls)) #appending every loss value to our loss list
		loss_mean += float(ls) #adding the value
        
		if(i%10==0): #we can comment this if the whole epoch is not taking very long
			print("Loss at iteration " + str(i+1) + " is " + str(float(ls)))
		i+=1 #comment this if we do not need iteration loss values displayed
        
	print('Training epoch takes {:.1f} sec'.format(time.time()-st_t)) #print current epoch time taken
    
	loss_mean /= len(X_train) #calculate the mean value

	# Validation
	st_t = time.time() #get the starting time for validation
    
	M_DL.eval() #define that nothing will be trained at this level
    
	loss_mean_valid = 0 
	v_loss_list = []
    
	for v_x, v_y in my_validation_batch_generator:
        
		v_x = torch.from_numpy(v_x) #same as training stage
		v_y = torch.from_numpy(v_y)
        
		if use_cuda:
			v_x = v_x.cuda(non_blocking=True) #same as training stage
			v_y = v_y.cuda(non_blocking=True)        
            
		v_ls = M_DL.get_loss(v_x, v_y).data.cpu().numpy()     #same as training stage 
        
		if(math.isnan(float(v_ls))): #same as training stage
			break
            
		v_loss_list.append(float(v_ls)) #same as training stage
		loss_mean_valid += float(v_ls)
        
	print('Validation takes {:.1f} sec'.format(time.time()-st_t))
	loss_mean_valid /= len(X_valid) #same as training stage


	f = open(fname, 'a') #opens our file in append stage, so that we do not overwrite it
	f.write('Epoch {}\ntrain loss mean: {}\nvalid loss mean: {}\n'.format(ep+1, loss_mean, loss_mean_valid)) #store the loss and validation mean
	print('Epoch {}\ntrain loss mean: {}\nvalid loss mean: {}\n'.format(ep+1, loss_mean, loss_mean_valid)) #display the same

	# Save model if the validation loss decreases
	check_interval = 1
	if loss_mean_valid < min_loss_v and ep % check_interval == 0: #this is to check per interval if the loss decreases and store it if it does
        
		min_loss_v = loss_mean_valid #change the minimum loss so that network compares against that value next epoch
		print('Save model at ep {}, mean of valid loss: {}'.format(ep+1, loss_mean_valid))  # print the value before saving
        
		torch.save(M_DL.state_dict(), fname+'_model.valid') #save the model
		torch.save(optimizer.state_dict(), fname+'_opt.valid') #save the optimizer
        
	# save if the training loss decrease
	check_interval = 1
	if loss_mean < min_loss_t and ep % check_interval == 0:
        
		min_loss_t = loss_mean #same as the validation step
		print('Save model at ep {}, mean of train loss: {}'.format(ep+1, loss_mean))
        
		torch.save(M_DL.state_dict(), fname+'_model.train') #same as the validation step
		torch.save(optimizer.state_dict(), fname+'_opt.train')
        
	f.close() #close the file, once written to

In [ ]:
from torchvision.utils import save_image #this library is needed to save the output we generate

In [ ]:
M_DL.eval() #tells network that we will just evaluate, and not to save anything
for img in X_test:
    
		inp = np.zeros((1,1,img_row,img_col),dtype=np.float32) #batch of 1 with 4 diemsional tensor since that is what the model expects

		name = img[46:len(img)-4] #get the name. Experiment and get the exact name to save against
		# print(name)
        
		ip = cv2.resize(cv2.imread(img, 0),(img_col,img_row)) #reshape to our needed input size
		cv2.imwrite('./data/test/'+(name) + '_ip.jpg', ip) #save the ip
        
		inp[0,0] = ip/255 #we normalize input to our needed shape and store in our array

		inp = torch.from_numpy(inp) #convert to numpy
        
		if use_cuda: #if using GPU
			inp = inp.cuda(non_blocking=True)
            
		output = M_DL(inp) #this will give us the normalized output
		output = output.data.cpu().numpy() #bring it back to CPU for writing to our disk and convert to numpy 

		cv2.imwrite('./data/test/'+(name) + '_op.jpg', output[0]*255) #save the output after normalizing back to 0-255 range